# Cell 2 cell game assignment

# Imports

#### Standard library imports

In [3]:
import sys
sys.path.append("../")

import os

#### Third party imports

In [4]:
import pandas as pd

#### Local application imports

In [7]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing data

In [8]:
data_path = '../pkg_dir/data/cell2cell.csv'

In [ ]:
dfr = pd.read_csv(data_path)

# Initial data wrangling

## Splitting train and test data

In [21]:
df_train = dfr[dfr['calibrat'] == 1].copy()
df_test = dfr[dfr['calibrat'] == 0].copy()

# Building models

#### Model features

In [25]:
m1f = [
    'customer',
    'calibrat',
    'churn',
    'churndep',
    'revenue',
    'mou',
    'recchrge',
    'directas',
    'overage',
    'roam',
    'changem',
    'changer',
    'dropvce',
    'blckvce',
    'unansvce',
    'custcare',
    'threeway',
    'mourec',
    'outcalls',
    'incalls',
    'peakvce',
    'opeakvce',
    'dropblk',
    'callfwdv',
    'callwait',
    'months',
    'uniqsubs',
    'actvsubs',
    'phones',
    'models',
    'eqpdays',
    'age1',
    'age2',
    'children',
    'credita',
    'creditaa',
    'prizmrur',
    'prizmub',
    'prizmtwn',
    'refurb',
    'webcap',
    'truck',
    'rv',
    'occprof',
    'occcler',
    'occcrft',
    'occstud',
    'occhmkr',
    'occret',
    'occself',
    'ownrent',
    'marryun',
    'marryyes',
    'mailord',
    'mailres',
    'mailflag',
    'travel',
    'pcown',
    'creditcd',
    'retcalls',
    'retaccpt',
    'newcelly',
    'newcelln',
    'refer',
    'incmiss',
    'income',
    'mcycle',
    'setprcm',
    'setprc',
    'retcall',
]

#### x

In [24]:
dfr.columns.tolist()

['customer',
 'calibrat',
 'churn',
 'churndep',
 'revenue',
 'mou',
 'recchrge',
 'directas',
 'overage',
 'roam',
 'changem',
 'changer',
 'dropvce',
 'blckvce',
 'unansvce',
 'custcare',
 'threeway',
 'mourec',
 'outcalls',
 'incalls',
 'peakvce',
 'opeakvce',
 'dropblk',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'phones',
 'models',
 'eqpdays',
 'age1',
 'age2',
 'children',
 'credita',
 'creditaa',
 'prizmrur',
 'prizmub',
 'prizmtwn',
 'refurb',
 'webcap',
 'truck',
 'rv',
 'occprof',
 'occcler',
 'occcrft',
 'occstud',
 'occhmkr',
 'occret',
 'occself',
 'ownrent',
 'marryun',
 'marryyes',
 'mailord',
 'mailres',
 'mailflag',
 'travel',
 'pcown',
 'creditcd',
 'retcalls',
 'retaccpt',
 'newcelly',
 'newcelln',
 'refer',
 'incmiss',
 'income',
 'mcycle',
 'setprcm',
 'setprc',
 'retcall']

# Header

# *Notes*

---

---